## 03. 다중선형회귀(Multivariable Linear Regression)

- 단순선형회귀(Simple Linear Regression): $x$가 1개인 선형회귀
- 다중선형회귀(Multivariable Linear Regression): 다수의 $x$로부터 $y$를 예측하는 선형회귀

$ H(x) = w_1x_1 + w_2x_2 + w_3x_3 + b $

(예) Quiz 세 번의 성적으로부터 기말 성적을 예측하는 모델을 만들고자 할 때

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
# 훈련 데이터 선언
x1_train = torch.FloatTensor([[73],
                              [93],
                              [89],
                              [96],
                              [73]])
x2_train = torch.FloatTensor([[80],
                              [88],
                              [91],
                              [98],
                              [66]])
x3_train = torch.FloatTensor([[75],
                              [93],
                              [90],
                              [100],
                              [70]])
y_train = torch.FloatTensor([[152],
                             [185],
                             [180],
                             [196],
                             [142]])

In [4]:
# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [5]:
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs):
    
    # H(x) 계산 = forward()
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b # 여기서는 일일이 x와 대응하는 w를 곱하도록 적어주었음 --> 개선예정!
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train)**2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이 누적되지 않도록 초기화해주고
    cost.backward() # 비용함수가 최솟값이 되는 지점의 parameter를 찾아야 하므로 비용함수에 대해 기울기 계산
    optimizer.step() # parameter 업데이트
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.294 w2: 0.294 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.674 w2: 0.661 w3: 0.676 b: 0.008 Cost: 1.563634
Epoch  200/1000 w1: 0.679 w2: 0.655 w3: 0.677 b: 0.008 Cost: 1.497607
Epoch  300/1000 w1: 0.684 w2: 0.649 w3: 0.677 b: 0.008 Cost: 1.435026
Epoch  400/1000 w1: 0.689 w2: 0.643 w3: 0.678 b: 0.008 Cost: 1.375730
Epoch  500/1000 w1: 0.694 w2: 0.638 w3: 0.678 b: 0.009 Cost: 1.319511
Epoch  600/1000 w1: 0.699 w2: 0.633 w3: 0.679 b: 0.009 Cost: 1.266222
Epoch  700/1000 w1: 0.704 w2: 0.627 w3: 0.679 b: 0.009 Cost: 1.215696
Epoch  800/1000 w1: 0.709 w2: 0.622 w3: 0.679 b: 0.009 Cost: 1.167818
Epoch  900/1000 w1: 0.713 w2: 0.617 w3: 0.680 b: 0.009 Cost: 1.122429


### 3. 벡터와 행렬 연산으로 바꾸기

- 기존에는 x값과 w값이 각각 3개여서 하나하나씩 정의해주고, 또 곱셈을 실행할 수 있었지만... input값과 가중치의 개수가 엄청나게 많다면?!
- 이를 해결하기 위해 행렬 곱셈 연산(또는 벡터의 내적)을 사용한다.
- 행렬의 곱셈 과정에서 이루어지는 벡터 연산(행x열)을 벡터의 내적(dot product)이라고 한다.

#### 행렬/벡터 연산으로 이해하기

$ H(X) = w_1x_1 + w_2x_2 + w_3x_3 $

위 식은 $(x_1 x_2 x_3) \cdot (w_1 w_2 w_3).T$ 처럼 벡터 간의 내적으로 표현할 수 있다.
두 벡터를 각각 $X$와 $W$로 표현한다면, 가설은 다음과 같다.

$ H(X) = XW $

즉, x와 w의 원소의 개수에 상관없이 두 개의 변수만으로 표현 가능!

$X$의 행렬 사이즈가 (5 x 3) (sample의 개수는 5개, feature는 3개)라고 한다면,  
$W$의 벡터 사이즈는 (3 x 1)이고,  
$XW$의 결과가 (5 x 1)이 되므로 편향 벡터 $B$의 사이즈 또한 (5 x 1)이 된다.

$ H(X) = XW + B$

### 4. 행렬 연산을 고려하여 파이토치로 구현하기

In [6]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 80],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152],
                             [185],
                             [180],
                             [196],
                             [142]])

In [8]:
print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [9]:
W = torch.zeros((3,1), requires_grad=True) # 행의 원소개수가 x_train의 열의 개수(feature의 개수)와 동일한 크기를 가져야 함
b = torch.zeros(1, requires_grad=True)

In [10]:
hypothesis = x_train.matmul(W) + b

In [12]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 80],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152],
                             [185],
                             [180],
                             [196],
                             [142]])

# 모델 초기화
W = torch.zeros((3,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs):
    
    # H(x) 계산
    # 편향 b는 브로드캐스팅되어 각 샘플에 더해진다.
    hypothesis = x_train.matmul(W) + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train)**2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([66.7178, 80.1701, 76.1025, 86.0194, 61.1565]) Cost: 9537.694336
Epoch    2/20 hypothesis: tensor([104.5421, 125.6208, 119.2478, 134.7862,  95.8280]) Cost: 3069.590088
Epoch    3/20 hypothesis: tensor([125.9858, 151.3882, 143.7087, 162.4333, 115.4844]) Cost: 990.670898
Epoch    4/20 hypothesis: tensor([138.1429, 165.9963, 157.5768, 178.1071, 126.6283]) Cost: 322.482086
Epoch    5/20 hypothesis: tensor([145.0350, 174.2780, 165.4395, 186.9928, 132.9461]) Cost: 107.717064
Epoch    6/20 hypothesis: tensor([148.9423, 178.9730, 169.8976, 192.0301, 136.5279]) Cost: 38.687496
Epoch    7/20 hypothesis: tensor([151.1574, 181.6346, 172.4254, 194.8856, 138.5585]) Cost: 16.499043
Epoch    8/20 hypothesis: tensor([152.4131, 183.1435, 173.8590, 196.5043, 139.7097]) Cost: 9.365656
Epoch    9/20 hypothesis: tensor([153.1250, 183.9988, 174.6723, 197.4217, 140.3625]) Cost: 7.071114
Epoch   10/20 hyp